In [1]:
drive_path = 'd:/'
import glob
#path = r'd:/BrainObservatory/ophys_analysis/'
file_path = r'd:/BrainObservatory/ophys_analysis/*.h5'
file_list = glob.glob(file_path)

import h5py
import os
import numpy as np
import pandas as pd

from allensdk.core.brain_observatory_cache import BrainObservatoryCache
manifest_path = os.path.join(drive_path,'BrainObservatory/manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_path)

In [2]:
#this chunk of code is to compute the CV for the OS and DS during drifting gratings. Because there are only half of the directions, 
#I think it makes sense to use exp1j1 instead of exp 1j2. Try both ways see what happens
#allPeakSFphs
file_path = r'd:/BrainObservatory/ophys_analysis/*.h5'
file_list = glob.glob(file_path)

orivals_rad = np.deg2rad([0,45,90,135,180,225,270,315])
cv_dgsMeanO=[]
cv_dgsMeanD=[]
cv_dgsMeanPO=[]
cv_dgsMeanPD=[]
drifting_csids=[]

for file in file_list:
    f = h5py.File(file)
    exp_id = file[35:44]
    if 'session_A' in file:
        data_set = boc.get_ophys_experiment_data(int(exp_id))
        csids = data_set.get_cell_specimen_ids()      
        response = f["analysis/response_dg"].value
        f.close()
        dg_means = response[:,1:,:-1,0]
        peak = pd.read_hdf(file, 'analysis/peak')
        TFpeak=(peak.tf_dg-1).values
        for i in range(dg_means.shape[2]):
            drifting_csids.append(csids[i])
            peakTFs=dg_means[:,TFpeak[i],i]
            meanTFs=np.ndarray.mean(dg_means[:,:,i],axis=1)
            tuning = meanTFs[:]
            tuning = np.where(tuning>0,tuning,0) #this is the fixed version that Saskia is computing now! don't forget 2theta for OS and 1 theta for DS
            tuningp = peakTFs[:]
            tuningp = np.where(tuningp>0,tuning,0)
            CV_topO = np.empty((8),dtype=np.complex128)
            CV_topD = np.empty((8),dtype=np.complex128)
            CV_topPO = np.empty((8),dtype=np.complex128)
            CV_topPD = np.empty((8),dtype=np.complex128)
            for i in range(8):
                CV_topO[i] = (tuning[i] * np.exp(1j * 2 * orivals_rad[i])).real
                CV_topD[i] = (tuning[i] * np.exp(1j * orivals_rad[i])).real
                CV_topPO[i] = (tuningp[i] * np.exp(1j * 2 * orivals_rad[i])).real
                CV_topPD[i] = (tuningp[i] * np.exp(1j * orivals_rad[i])).real
            cv_dgsMeanO.append(np.abs(CV_topO.sum() / tuning.sum()))
            cv_dgsMeanD.append(np.abs(CV_topD.sum() / tuning.sum()))
            cv_dgsMeanPO.append(np.abs(CV_topPO.sum() / tuningp.sum()))
            cv_dgsMeanPD.append(np.abs(CV_topPD.sum() / tuningp.sum()))
#             allcv_sgsO.append(cv_sgsO)
#             allcv_sgsD.append(cv_sgsD)

In [5]:
# file=r'D:\BrainObservatory\ophys_analysis\497256116_three_session_B_analysis.h5'
# f = h5py.File(file)
# f['analysis'].keys()
# #response = f["analysis/response_sg"].value

In [4]:
#same as above but for static
file_path = r'd:/BrainObservatory/ophys_analysis/*.h5'
#file_dir=r'd:/BrainObservatory/ophys_analysis/'
file_list = glob.glob(file_path)
#file_list=os.listdir(file_dir)

cv_sgs_peaks=[]
cv_sgs_pSF=[]
cv_sgs_pPH=[]
cv_sgs_means=[]
orivals_rad = np.deg2rad([0,30,60,90,120,150])

allcsids=[]
a=[]
for file in file_list:
    f = h5py.File(file)
    #f=h5py.File(os.path.join(file_dir,file))
    exp_id = file[35:44]
    if 'session_B' in file:
        data_set = boc.get_ophys_experiment_data(int(exp_id))
        csids = data_set.get_cell_specimen_ids()
        response = f["analysis/response_sg"].value
        f.close()
        sg_means = response[:,1:,:,:-1,0]
        peak = pd.read_hdf(file, 'analysis/peak')
        SFpeak=(peak.sf_sg-1).values
        PHpeak=(peak.phase_sg).values
        for i in range(sg_means.shape[3]):
            allcsids.append(csids[i])
            peakSFPHs=sg_means[:,SFpeak[i],PHpeak[i],i]
            peakSFmPHs=np.ndarray.mean(sg_means[:,SFpeak[i],:,i],axis=1)
            peakPHmSFs=np.ndarray.mean(sg_means[:,:,PHpeak[i],i],axis=1)
            means=np.ndarray.mean(sg_means[:,:,:,i],axis=1)
            means=np.ndarray.mean(means[:,:],axis=1)
            #tuning = peaks[:]
            peakSFPHs = np.where(peakSFPHs>0,peakSFPHs,0) #don't forget 2theta for OS and 1 theta for DS
            peakSFmPHs = np.where(peakSFmPHs>0,peakSFmPHs,0)
            peakPHmSFs = np.where(peakPHmSFs>0,peakPHmSFs,0)
            means = np.where(means>0,means,0)
            CV_top = np.empty((6),dtype=np.complex128)
            CV_topSFmP = np.empty((6),dtype=np.complex128)
            CV_topPmSF = np.empty((6),dtype=np.complex128)
            CV_m = np.empty((6),dtype=np.complex128)
            for i in range(6):
                CV_top[i] = (peakSFPHs[i] * np.exp(1j * orivals_rad[i])).real
                CV_topSFmP[i] = (peakSFmPHs[i] * np.exp(1j * orivals_rad[i])).real
                CV_topPmSF[i] = (peakPHmSFs[i] * np.exp(1j * orivals_rad[i])).real
                CV_m[i] = (means[i] * np.exp(1j * orivals_rad[i])).real
            cv_sgs_peaks.append(np.abs(CV_top.sum() / peakSFPHs.sum()))
            cv_sgs_pSF.append(np.abs(CV_topSFmP.sum() / peakSFmPHs.sum()))
            cv_sgs_pPH.append(np.abs(CV_topPmSF.sum() / peakPHmSFs.sum()))
            cv_sgs_means.append(np.abs(CV_m.sum() / means.sum()))
            
        #allcv_sgs.append(cv_sgs)

In [32]:
DG_data_dict = {
    'cv_dgsMeanO':cv_dgsMeanO,
    'cv_dgsMeanD':cv_dgsMeanD,
    'cv_dgsPeakO':cv_dgsMeanPO,
    'cv_dgsPeakD':cv_dgsMeanPD,
    'drifting_csids':drifting_csids
    }

SG_data_dict = {
    'cv_sgs_peaks':cv_sgs_peaks,
    'cv_sgs_pSF':cv_sgs_pSF,
    'cv_sgs_pPH':cv_sgs_pPH,
    'cv_sgs_means':cv_sgs_means,
    'static_csids':allcsids
    }

In [47]:
np.save('SG_data_dict',SG_data_dict)

In [48]:
np.save('DG_data_dict',DG_data_dict)

In [49]:
import pickle
pickle.dump(DG_data_dict, open("DG_data_dict.p","wb"))
pickle.dump(SG_data_dict, open("SG_data_dict.p","wb"))